# Query Transformations

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", max_tokens=1000)

#Query Rewriter

In [3]:
from langchain_core.prompts import PromptTemplate
import nest_asyncio
nest_asyncio.apply()

import asyncio

rewriter_prompt = """
    You are an AI assistant, Your task is to reformulate new queries for RAG syatem for better context retrieval.
    Given the original query, rewrite it into more specific, detailed and likely to retrieve relevent information 
    Original_Query: {query}
    Rewritten_Query:
"""

query_rewrite_prompt = PromptTemplate(
    input_variables=["query"],
    template = rewriter_prompt
)

query_rewriter = query_rewrite_prompt | llm

async def rewrite_query(original_query):
    """
    Rewrite the original query to improve retrieval.
    
    Args:
    original_query (str): The original user query
    
    Returns:
    str: The rewritten query
    """
    response = await query_rewriter.ainvoke(original_query)
    return response.content

if __name__ == "__main__":
    original_query = "What are the impacts of climate change on the environment?"
    print(asyncio.run(rewrite_query(original_query)))
    #print("Original query:", original_query)
    #print("\nRewritten query:", rewritten_query)

Based on the original query, I would rewrite it to:

"What are the specific physical, biological, and ecological impacts of climate change on the environment, including changes in temperature, precipitation, sea-level rise, and extreme weather events, and how do these impacts affect ecosystems, biodiversity, and human settlements?"

Rewritten query highlights:

1. **Specificity**: It focuses on specific aspects of climate change, such as temperature, precipitation, sea-level rise, and extreme weather events, making it more targeted for RAG system.
2. **Context**: It adds context by mentioning the impact on ecosystems, biodiversity, and human settlements, making it more relevant and likely to retrieve specific information.
3. **Technical terms**: Incorporating technical terms like "physical, biological, and ecological impacts" can help RAG system retrieve more accurate and detailed information.
4. **Clarity**: The rewritten query is more concise and clear, reducing ambiguity and increas

#Step-back Prompting

In [4]:
step_back_template = """You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.

Original query: {original_query}

Step-back query:"""

step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_template
)

step_back_chain = step_back_prompt | llm

async def generate_step_back_query(original_query):
    """
    Generate a step-back query to retrieve broader context.
    
    Args:
    original_query (str): The original user query
    
    Returns:
    str: The step-back query
    """
    response = await step_back_chain.ainvoke(original_query)
    return response.content

if __name__ == "__main__":
    original_query = "What are the impacts of climate change on the environment?"
    print(asyncio.run(generate_step_back_query(original_query)))

Based on the original query, a step-back query that is more general and can help retrieve relevant background information could be:

"What are the main factors that affect the environment?"

This step-back query is more general because it:

1. Shifts the focus from a specific topic (climate change) to a broader category (environmental factors).
2. Encourages the retrieval of background information on various environmental factors, which can provide a better understanding of the context surrounding climate change.

By generating this more general query, the RAG system can retrieve relevant information on various environmental factors, such as pollution, deforestation, population growth, and human activity, which can help provide a more comprehensive understanding of the issue.


#Sub-Query Decomposition

In [9]:
subquery_decomposition_template = """
    You are ans AI assistant tasked with decomposing the original query into sub queries for better retrieval in RAG system.
    Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.
    Original Query: {original_query}
    
    example: What are the impacts of climate change on the environment?
    Sub-queries:
    1. What are the impacts of climate change on biodiversity?
    2. How does climate change affect the oceans?
    3. What are the effects of climate change on agriculture?
    4. What are the impacts of climate change on human health?

    Only return questions, DO NOT provide any additional explanation.
"""

subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

subquery_decomposer_chain = subquery_decomposition_prompt | llm

async def decompose_query(original_query: str):
    """
    Decompose the original query into simpler sub-queries.
    
    Args:
    original_query (str): The original complex query
    
    Returns:
    List[str]: A list of simpler sub-queries
    """
    response = await subquery_decomposer_chain.ainvoke(original_query)
    
    return response

original_query = "What are the impacts of ai in human life?"
sub_queries = asyncio.run(decompose_query(original_query))
response = sub_queries.content
sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
Here are the decomposed sub-queries for the original query:
1. What are the effects of AI on human relationships?
2. How does AI influence job market and employment?
3. What are the impacts of AI on education and learning?
4. How does AI affect healthcare and medical services?
